In [1]:
import os
os.chdir('/data/users/jiancheng.zhai/project/wmd/doc')
import sys
sys.path.append("../utils")
sys.path.append("../tools/word_movers_distance")
import w2v_embeddings
import WMD_pyemd
import numpy as np
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cross_validation import train_test_split
import read_conf
import pandas as pd

In [99]:
import time

In [2]:
from time import time
import pickle
import sys
import pdb

import numpy as np

from pyemd import emd

import logging

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

def nBOW(document, vocab):
    '''
    Compute nBOW representation of a document

    Input:
    doc:        Document, list of words (strings).
    vocab:      Set of words in all documents.

    Returns:    List of floats, nBow representation.
    '''

    doc_len = len(document)
    d = []
    for i, t in enumerate(vocab):
        d.append(document.count(t) / float(doc_len))

    return d


In [25]:
#set(d1 + d2)
#np.array()
#d1.append()

#nBOW(d1, set(d1+d2))
#nBOW(d2, set(d1+d2))
vl = len(vocab_list)
dm = np.zeros((vl, vl), dtype=np.float)
dm.shape

for i, t1 in enumerate(vocab_list):
    for j, t2 in enumerate(vocab_list):
        dm[i][j] = np.sqrt(np.sum((model[t1] - model[t2])**2))

In [41]:
sdm = dm[[1,3,4]]

In [43]:
sdm[[1,2,3]]

IndexError: index 3 is out of bounds for axis 0 with size 3

In [3]:
def WMD(document1, document2, model):
    '''
    Compute WMD.

    Input:
    document1:      List of words.
    document2:      List of words.
    model:          Word2vec model, providing the word embeddings.
        vocab:          Set of words in all documents.

    Returns:        WMD between documents, float.
    '''

    # Remove out-of-vocabulary words.
    len_pre_oov1 = len(document1)
    len_pre_oov2 = len(document2)
#    document1 = [token for token in document1 if token in model.vocab.keys()]
#    document2 = [token for token in document2 if token in model.vocab.keys()]
#    logging.info('Removed %d and %d OOV words from document 1 and 2 (respectively).', len_pre_oov1 - len(document1), len_pre_oov2 - len(document2))

    if len(document1) == 0 or len(document2) == 0:
        logging.info('At least one of the documents had no words that were in the vocabulary. Aborting (returning NaN).')
        return float('nan')

    vocab = set(document1 + document2)

    # Compute nBOW representation of documents.
    d1 = np.array(nBOW(document1, vocab))
    d2 = np.array(nBOW(document2, vocab))

    vocab_len = len(vocab)
    distance_matrix = np.zeros((vocab_len, vocab_len), dtype=np.float)
    for i, t1 in enumerate(vocab):
        for j, t2 in enumerate(vocab):
            if not t1 in document1 or not t2 in document2:
                # Only compute the distances that we need.
                continue
            # Compute Euclidean distance between word vectors.
            # TODO: this matrix is (and should be) symmetric, so we can save some computation here.
            # TODO: why not cosine distance?
            distance_matrix[i][j] = np.sqrt(np.sum((model[t1] - model[t2])**2))

    # Return WMD.
    return emd(d1, d2, distance_matrix)



In [5]:
from gensim.models.word2vec import Word2Vec
model = Word2Vec.load("../data/raw.model")


In [6]:
with open("../data/embed.vocab") as f:
    vocab_list = map(str.strip, f.readlines())

In [7]:
vocab_list

['juxiangyou_com',
 'game_juxiangyou_com',
 'eloancn_com',
 'pangpangpig_com',
 'dhqjr_com',
 'hx9999_com',
 'cgtz_com',
 'yrw_com',
 'app_etongdai_com',
 'touna_cn',
 '91jin_com',
 '99fan_com',
 'wsloan_com',
 'haochedai_com',
 'user_lu_com',
 'gaosouyi_com',
 'trjcn_com',
 'frbao_com',
 'jfz_com',
 'itoumi_com',
 'zhitouwang_cn',
 'yuntongxun_com',
 'huiyingdai_com',
 'cnaidai_com',
 'chenghuitong_net',
 'yinker_com',
 'yinhu_com',
 'old_jinshangdai_com',
 'zhaoshangdai_com',
 'xinhehui_com',
 'hrcf168_com',
 'etongdai_com',
 'edai_com',
 'haoinvest_com',
 'baijindai_com',
 'd_com_cn',
 '9rjr_com',
 'xrkdai_com',
 'ma_juxiangyou_com',
 'lejinsuo_com',
 '360taihe_com',
 '1001_cn',
 'sslj8_cn',
 'qtyd_com',
 'jimengcaifu_com',
 'hitouba_com',
 'gkkxd_com',
 'tuandai_com',
 'fanli_juxiangyou_com',
 'fa_163_com',
 'v_51juejinjie_com',
 'ftoul_com',
 'bbs_wsloan_com',
 'fuyph_com',
 'yibaijin_com',
 'xiangmu_trjcn_com',
 'jia16_com',
 'xiaozhu168_com',
 'bjdp2p_com',
 'fangyuano_com',
 'q

In [44]:
comp_domain_file = "../data/comp_url_data"
comp_dict = {}

fd = open(comp_domain_file)
for line in fd:
    line.strip()
    arr = line.split('\t')
    comp_id = arr.pop(0)
    comp_dict[comp_id] = arr
fd.close()



In [45]:

comp_list = comp_dict.keys()

for comp_id in comp_list:
    s = comp_dict[comp_id]
    s = [x for x in s if x in vocab_list]
    comp_dict[comp_id] = s
    if(len(s) < 100000):
        del comp_dict[comp_id]

print len(comp_dict.keys())

51


In [10]:
d1 = "app_etongdai_com touna_cn 99fan_com haochedai_com".split()
d2 = "yuntongxun_com huiyingdai_com yinhu_com zhaoshangdai_com".split()

In [12]:
D = WMD(d1, d2, model)
D

6.251723528767347

In [46]:
s1 = comp_dict['1845']
s2 = comp_dict['2814']

In [48]:
len(s1)
len(s2)

566765

In [51]:
#WMD(s1, s2, model)
v = set(s1+s2)

In [53]:
a1 = np.array(nBOW(s1, v))
a2 = np.array(nBOW(s2, v))


In [54]:
len(v)

70

In [56]:
distance_matrix = np.zeros((len(v), len(v)), dtype=np.float)

In [59]:
vob_index_dict = {}
for i, vob in enumerate(vocab_list):
    vob_index_dict[vob] = i

In [73]:

u1

{'10_0_3_121',
 '10_0_3_151',
 '10_0_3_50',
 '117_71_18_12',
 '127_0_0_1',
 '183_61_9_9',
 '192_168_1_216',
 '220_115_251_229',
 '61_240_234_8',
 'ah_189_cn',
 'ankuocaifu_com',
 'baidu_com',
 'bbs_touna_cn',
 'ckmap_mediav_com',
 'dhqjr_com',
 'eiv_baidu_com',
 'eloancn_com',
 'game_juxiangyou_com',
 'gkkxd_com',
 'gtm-msr_appspot_com',
 'hao_360_cn',
 'haolawyers_cn',
 'hi_jsinfo_net',
 'hitouba_com',
 'hx9999_com',
 'jrh_touna_cn',
 'juxiangyou_com',
 'm_baidu_com',
 'm_quanmama_com',
 'm_touna_cn',
 'my_touna_cn',
 'new_touna_cn',
 'newunion_cn',
 'old_touna_cn',
 'oneshop_tonace_vm',
 'oneshop_touna_cn',
 'p_yiqifa_com',
 'pangpangpig_com',
 'pinzhuan_baidu_com',
 'quanmama_com',
 'sogou_com',
 'sslj8_cn',
 'tongji_baidu_com',
 'touna_cn',
 'tounahd_com',
 'tounawang_ewdtx_com',
 'trjcn_com',
 'user_lu_com',
 'v_51juejinjie_com',
 'yinhu_com',
 'yuntongxun_com',
 'zcmall_com'}

In [77]:
il1 = map(lambda x: vob_index_dict[x], d1)
il2 = map(lambda x: vob_index_dict[x], d2)

In [100]:
def calc_wmd(d1, d2, dm, vob_index_dict):
    
    u1 = set(d1)
    u2 = set(d2)
    du = u1.union(u2)
    
    f1 = np.array(nBOW(d1, du))
    f2 = np.array(nBOW(d2, du))


    dul = len(du)
    dum = np.zeros((dul, dul), dtype=np.float)
    du_list = list(du)
    processed_list = []
    for i, t1 in enumerate(du_list):
        processed_list.append(i)
        for j, t2 in enumerate(du_list):
            if j in processed_list:
                continue
            dist_matrix_x = vob_index_dict[t1]
            dist_matrix_y = vob_index_dict[t2]
            dist = dm[dist_matrix_x, dist_matrix_y]
         
            #np.sqrt(np.sum((model[t1] - model[t2])**2))
            dum[i][j] = dist
            dum[j][i] = dist

    return emd(f1, f2, dum)

In [98]:
s1 = comp_dict['1845']
s2 = comp_dict['2814']

In [101]:
start = time.clock()

calc_wmd(s1, s2, dm,  vob_index_dict)

#WMD(s1, s2, model)

elapsed = (time.clock() - start)
print("Time used:",elapsed)

('Time used:', 1.8000000000000114)


In [102]:
start = time.clock()

#calc_wmd(s1, s2, dm,  vob_index_dict)

WMD(s1, s2, model)

elapsed = (time.clock() - start)
print("Time used:",elapsed)

('Time used:', 332.59000000000003)


In [ ]:
res_file = "../data/comp_sim_res"
wfd = open(res_file, 'w+')

processed_comp_list = []
for comp1 in comp_dict:
    processed_comp_list.append(comp1)
    for comp2 in comp_dict:
        if comp2 in processed_comp_list:
            continue
            
        dist = calc_wmd(comp_dict[comp1], comp_dict[comp2], dm, vob_index_dict)
        out = str(comp1) + "\t" + str(comp2) + "\t" + str(dist) + "\n"
        wfd.write(out)
        print out
wfd.close()

3719	1705	7.0826553566

3719	1127	8.17688475419

3719	2543	5.50398577908

3719	1232	6.09210999313

3719	1845	6.61788779544

3719	4101	5.99272511423

3719	4105	6.59532425907

3719	1178	7.21147699123

3719	1572	7.99416455645

3719	1570	6.46851339474

3719	4986	6.41380068681

3719	4038	5.46359235123

3719	3843	6.6049083908

3719	5495	6.8688646269

3719	2272	6.68305296505

3719	1876	8.01668627359

3719	4946	5.37246666504

3719	3694	6.96822166946

3719	2025	6.78990270231

3719	3607	6.93436070858

3719	3350	7.30733732518

3719	2527	6.13627086357

3719	3122	6.6837380042

3719	3405	7.50211958918

3719	3448	6.5662323909

3719	4120	6.5038277407

3719	2936	7.03010902246

3719	1209	6.68473865141

3719	1514	7.12340055621

3719	2740	8.07356762924

3719	3222	7.34443777163

3719	2320	5.83895203236

3719	4561	9.92631546628

3719	2392	7.25861027034

3719	2257	6.69439007337

3719	2772	6.84999117

3719	5155	6.53457409359

3719	3811	7.26595372748

3719	2600	5.65426274547

3719	3951	7.72401601993

3719	1423

In [105]:
distance_matrix_file = "../data/comp_distance_matrix"


hi
